In [67]:
import warnings
warnings.filterwarnings('ignore')


In [68]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split,KFold
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as n
from mypipes import *

In [69]:
datafile_train=r'/Users/nikita/Downloads/Project1/P1 Data/Consumer_Complaints_train.csv'
datafile_test=r'/Users/nikita/Downloads/Project1/P1 Data/Consumer_Complaints_test_share.csv'
cd_train=pd.read_csv(datafile_train)
cd_test=pd.read_csv(datafile_test)

In [70]:
cd_train.shape


(478421, 18)

In [71]:
cd_test.shape

(119606, 17)

In [72]:
cd_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478421 entries, 0 to 478420
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 478421 non-null  object
 1   Product                       478421 non-null  object
 2   Sub-product                   339948 non-null  object
 3   Issue                         478421 non-null  object
 4   Sub-issue                     185796 non-null  object
 5   Consumer complaint narrative  75094 non-null   object
 6   Company public response       90392 non-null   object
 7   Company                       478421 non-null  object
 8   State                         474582 non-null  object
 9   ZIP code                      474573 non-null  object
 10  Tags                          67206 non-null   object
 11  Consumer consent provided?    135487 non-null  object
 12  Submitted via                 478421 non-null  object
 13 

In [73]:
import sklearn as skl

In [74]:
print(skl.__version__)

1.3.0


In [75]:
cd_train.head(20)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572
5,2014-06-26,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Bank of America,WI,54313,NaN,NaN,Referral,2014-07-01,Closed with explanation,Yes,No,913769
6,2012-09-28,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Ocwen,FL,33168,NaN,NaN,Referral,2012-10-01,Closed with explanation,Yes,No,163903
7,2015-05-06,Mortgage,FHA mortgage,"Loan modification,collection,foreclosure",NaN,I have a mortgage with Quicken Loans. From XXX...,NaN,Quicken Loans,GA,300XX,NaN,Consent provided,Web,2015-05-06,Closed with explanation,Yes,Yes,1362945
8,2013-02-25,Credit card,NaN,Credit card protection / Debt protection,NaN,NaN,NaN,Discover,OH,45640,NaN,NaN,Web,2013-02-26,Closed with monetary relief,Yes,No,331502
9,2016-03-30,Bank account or service,Checking account,Deposits and withdrawals,NaN,I put my settlement check in to bank of americ...,Company has responded to the consumer and the ...,Bank of America,TX,787XX,NaN,Consent provided,Web,2016-04-01,Closed with explanation,Yes,No,1855737


In [76]:
cd_train['Issue'].unique()


array(['Billing statement', 'Making/receiving payments, sending money',
       'Incorrect information on credit report', 'Transaction issue',
       'Loan modification,collection,foreclosure',
       'Loan servicing, payments, escrow account',
       'Credit card protection / Debt protection',
       'Deposits and withdrawals',
       "Cont'd attempts collect debt not owed", 'Getting a loan', 'Other',
       'Account opening, closing, or management',
       "Credit reporting company's investigation",
       'Improper use of my credit report', 'Managing the loan or lease',
       'Communication tactics', 'False statements or representation',
       'Dealing with my lender or servicer',
       'Problems caused by my funds being low', 'Delinquent account',
       'Identity theft / Fraud / Embezzlement',
       'Other transaction issues', 'Disclosure verification of debt',
       'Balance transfer fee', 'Using a debit or ATM card',
       'Improper contact or sharing of info',
       'Appl

In [77]:
cd_train.sample(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
403314,2014-01-07,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,NaN,NaN,North American Credit Services,FL,33525,NaN,NaN,Postal mail,2014-02-06,Closed with explanation,Yes,No,657374
309181,2015-08-25,Mortgage,Home equity loan or line of credit,"Loan servicing, payments, escrow account",NaN,NaN,Company chooses not to provide a public response,Bank of America,NJ,08558,NaN,NaN,Referral,2015-08-30,Closed with explanation,Yes,Yes,1536220
2635,2014-04-11,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,GA,30012,NaN,NaN,Phone,2014-04-15,Closed with explanation,Yes,No,804792
266436,2014-02-28,Debt collection,Credit card,Cont'd attempts collect debt not owed,Debt was paid,NaN,NaN,Citibank,IL,62939,NaN,NaN,Phone,2014-03-05,Closed with explanation,Yes,No,737464
253127,2014-08-26,Consumer Loan,Vehicle loan,Problems when you are unable to pay,NaN,NaN,NaN,Wells Fargo & Company,MD,21212,Older American,NaN,Web,2014-08-26,Closed with explanation,Yes,No,1001652
116456,2015-04-13,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,My ex-wife has charges from ministry health ca...,Company chooses not to provide a public response,"Alliance Collection Agencies, Inc.",WI,544XX,NaN,Consent provided,Web,2015-04-13,Closed with explanation,Yes,Yes,1327045
161253,2012-12-21,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,PNC Bank N.A.,GA,30253,NaN,NaN,Referral,2012-12-26,Closed with explanation,Yes,No,215273
168327,2016-04-21,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,Company has responded to the consumer and the ...,Experian,IL,60446,NaN,Consent not provided,Web,2016-04-25,Closed with non-monetary relief,Yes,Yes,1889487
405731,2013-10-10,Credit card,NaN,Identity theft / Fraud / Embezzlement,NaN,NaN,NaN,HSBC North America Holdings Inc.,NY,10704,NaN,NaN,Referral,2013-10-16,Closed with monetary relief,Yes,No,557336
104190,2012-05-09,Credit card,NaN,Billing disputes,NaN,NaN,NaN,Capital One,IL,60062,NaN,NaN,Referral,2012-05-09,Closed without relief,Yes,Yes,73957


In [78]:
cd_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478421 entries, 0 to 478420
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 478421 non-null  object
 1   Product                       478421 non-null  object
 2   Sub-product                   339948 non-null  object
 3   Issue                         478421 non-null  object
 4   Sub-issue                     185796 non-null  object
 5   Consumer complaint narrative  75094 non-null   object
 6   Company public response       90392 non-null   object
 7   Company                       478421 non-null  object
 8   State                         474582 non-null  object
 9   ZIP code                      474573 non-null  object
 10  Tags                          67206 non-null   object
 11  Consumer consent provided?    135487 non-null  object
 12  Submitted via                 478421 non-null  object
 13 

In [79]:
from mypipes import *

In [80]:
p1=pdPipeline([
    ('var_select',VarSelector(['Date sent to company','Date received'])),
    ('convert_to_datetime',Convert_to_datetime()),
    ('date_diff_days',date_diff_days()),
     ('missing_trt',DataFrameImputer())
])

In [81]:
cd_train[['Date sent to company','Date received']]

,Date sent to company,Date received
0,2014-05-16,2014-05-15
1,2014-09-24,2014-09-18
2,2014-04-03,2014-03-13
3,2015-07-17,2015-07-17
4,2014-11-28,2014-11-20
...,...,...
478416,2015-07-01,2015-06-30
478417,2013-06-21,2013-06-20
478418,2014-09-09,2014-09-09
478419,2012-09-24,2012-09-21


In [82]:
k=cd_test['State'].value_counts()
print(k)

CA    17498
FL    11676
TX     8958
NY     8469
GA     5410
      ...  
FM        5
MH        3
AS        3
AA        3
PW        2
Name: State, Length: 62, dtype: int64


In [83]:
p2=pdPipeline([
     ('var_select',VarSelector(['Sub-product','Sub-issue','Consumer complaint narrative',
            'Company public response','Tags','Consumer consent provided?'])),
      ('convert_to_cyclic',convert_to_cyclic()),
    ('missing_trt',DataFrameImputer())
    ])

In [84]:
p3=pdPipeline([
     ('var_select',VarSelector(['Issue'])),
     ('convert_to',convert_to_cyclic1()),
     ('missing_trt',DataFrameImputer())
     ])

In [85]:
p4=pdPipeline([
     ('var_select',VarSelector(['State'])),
     ('convert_to_dummy2',convert_to_cyclic2(x=k)),
     ('missing_trt',DataFrameImputer())
     ])

In [86]:
p5=pdPipeline([
     ('var_select',VarSelector(['Product','Submitted via','Company response to consumer','Timely response?'])),
     ('convert_dummy',get_dummies_Pipe1()),
     ('missing_trt',DataFrameImputer())
     ])

In [87]:
data_pipe=FeatureUnion([
    ('obj_to_date',p1),
    ('get_test',p2),
    ('get_test1',p3),
    ('get_test2',p4),
    ('get_dummy',p5)
])

In [88]:
data_pipe.fit(cd_train)

Loan modification,collection,foreclosure
Incorrect information on credit report
Loan servicing, payments, escrow account
Cont'd attempts collect debt not owed
Account opening, closing, or management
Disclosure verification of debt
Communication tactics
Deposits and withdrawals
Application, originator, mortgage broker
Billing disputes
CA
FL
TX
NY
GA
NJ
PA
IL
VA
MD
OH
NC
MI
AZ
WA


FeatureUnion(transformer_list=[('obj_to_date',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['Date '
                                                                              'sent '
                                                                              'to '
                                                                              'company',
                                                                              'Date '
                                                                              'received'])),
                                                  ('convert_to_datetime',
                                                   Convert_to_datetime()),
                                                  ('date_diff_days',
                                                   date_diff_days()),
                                                  ('missing_trt',
                                                   DataFrameImputer())])),
                               ('get_test',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['Sub-pro...
                                                   convert_to_cyclic2(x=CA    17498
FL    11676
TX     8958
NY     8469
GA     5410
      ...  
FM        5
MH        3
AS        3
AA        3
PW        2
Name: State, Length: 62, dtype: int64)),
                                                  ('missing_trt',
                                                   DataFrameImputer())])),
                               ('get_dummy',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['Product',
                                                                              'Submitted '
                                                                              'via',
                                                                              'Company '
                                                                              'response '
                                                                              'to '
                                                                              'consumer',
                                                                              'Timely '
                                                                              'response?'])),
                                                  ('convert_dummy',
                                                   get_dummies_Pipe1()),
                                                  ('missing_trt',
                                                   DataFrameImputer())]))])

In [89]:
data_pipe.transform(cd_train).shape


Loan modification,collection,foreclosure
Incorrect information on credit report
Loan servicing, payments, escrow account
Cont'd attempts collect debt not owed
Account opening, closing, or management
Disclosure verification of debt
Communication tactics
Deposits and withdrawals
Application, originator, mortgage broker
Billing disputes
CA
FL
TX
NY
GA
NJ
PA
IL
VA
MD
OH
NC
MI
AZ
WA


(478421, 55)

In [90]:
data_pipe.get_feature_names_out()


array(['obj_to_date__date_diff', 'get_test__Sub_product_isNan',
       'get_test__Sub_issue_isNan',
       'get_test__Consumer_complaint_narrative_isNan',
       'get_test__Company_public_response_isNan', 'get_test__Tags_isNan',
       'get_test__Consumer_consent_provided_isNan',
       'get_test1__Issue_Loan_modification_collection_foreclosure',
       'get_test1__Issue_Incorrect_information_on_credit_report',
       'get_test1__Issue_Loan_servicing__payments__escrow_account',
       "get_test1__Issue_Cont'd_attempts_collect_debt_not_owed",
       'get_test1__Issue_Account_opening__closing__or_management',
       'get_test1__Issue_Disclosure_verification_of_debt',
       'get_test1__Issue_Communication_tactics',
       'get_test1__Issue_Deposits_and_withdrawals',
       'get_test1__Issue_Application__originator__mortgage_broker',
       'get_test1__Issue_Billing_disputes', 'get_test2__State_CA',
       'get_test2__State_FL', 'get_test2__State_TX',
       'get_test2__State_NY', 'get_te

In [91]:
data_pipe.get_feature_names_out()


array(['obj_to_date__date_diff', 'get_test__Sub_product_isNan',
       'get_test__Sub_issue_isNan',
       'get_test__Consumer_complaint_narrative_isNan',
       'get_test__Company_public_response_isNan', 'get_test__Tags_isNan',
       'get_test__Consumer_consent_provided_isNan',
       'get_test1__Issue_Loan_modification_collection_foreclosure',
       'get_test1__Issue_Incorrect_information_on_credit_report',
       'get_test1__Issue_Loan_servicing__payments__escrow_account',
       "get_test1__Issue_Cont'd_attempts_collect_debt_not_owed",
       'get_test1__Issue_Account_opening__closing__or_management',
       'get_test1__Issue_Disclosure_verification_of_debt',
       'get_test1__Issue_Communication_tactics',
       'get_test1__Issue_Deposits_and_withdrawals',
       'get_test1__Issue_Application__originator__mortgage_broker',
       'get_test1__Issue_Billing_disputes', 'get_test2__State_CA',
       'get_test2__State_FL', 'get_test2__State_TX',
       'get_test2__State_NY', 'get_te

In [92]:
x_train=pd.DataFrame(data=data_pipe.fit_transform(cd_train),
                     columns=data_pipe.get_feature_names_out())

Loan modification,collection,foreclosure
Incorrect information on credit report
Loan servicing, payments, escrow account
Cont'd attempts collect debt not owed
Account opening, closing, or management
Disclosure verification of debt
Communication tactics
Deposits and withdrawals
Application, originator, mortgage broker
Billing disputes
CA
FL
TX
NY
GA
NJ
PA
IL
VA
MD
OH
NC
MI
AZ
WA


In [93]:
x_test=pd.DataFrame(data=data_pipe.fit_transform(cd_test),
                     columns=data_pipe.get_feature_names_out())

Loan modification,collection,foreclosure
Incorrect information on credit report
Loan servicing, payments, escrow account
Cont'd attempts collect debt not owed
Account opening, closing, or management
Disclosure verification of debt
Communication tactics
Deposits and withdrawals
Application, originator, mortgage broker
Billing disputes
CA
FL
TX
NY
GA
NJ
PA
IL
VA
MD
OH
NC
MI
AZ
WA


In [94]:
x_train.shape

(478421, 55)

In [95]:
x_test.shape


(119606, 55)

In [96]:
x_test.sample(4)


,obj_to_date__date_diff,get_test__Sub_product_isNan,get_test__Sub_issue_isNan,get_test__Consumer_complaint_narrative_isNan,get_test__Company_public_response_isNan,get_test__Tags_isNan,get_test__Consumer_consent_provided_isNan,get_test1__Issue_Loan_modification_collection_foreclosure,get_test1__Issue_Incorrect_information_on_credit_report,get_test1__Issue_Loan_servicing__payments__escrow_account,...,get_dummy__Product_Credit card,get_dummy__Product_Credit reporting,get_dummy__Product_Debt collection,get_dummy__Product_Money transfers,get_dummy__Product_Mortgage,get_dummy__Product_Other financial service,get_dummy__Product_Payday loan,get_dummy__Product_Prepaid card,get_dummy__Product_Student loan,get_dummy__Product_Virtual currency
66224,0,0,0,1,1,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
101325,7,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21996,2,0,0,1,1,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
75533,0,1,1,1,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [97]:
k1=cd_train['State'].nunique()
print(k1)
k=cd_test['State'].value_counts()
print(k)


62
CA    17498
FL    11676
TX     8958
NY     8469
GA     5410
      ...  
FM        5
MH        3
AS        3
AA        3
PW        2
Name: State, Length: 62, dtype: int64


In [98]:
y_train=cd_train['Consumer disputed?']

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [100]:
lr=LogisticRegression()

In [101]:
lr.fit(x_train,y_train)

LogisticRegression()

In [102]:
cv_mae=cross_val_score(lr,
                      x_train,y_train,
                      cv=10,
                      scoring='neg_mean_absolute_error')

In [108]:
cv_mae


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [109]:
cv_mae.mean()

nan

In [110]:
test = lr.predict(x_test)
print(test)

['No' 'No' 'No' ... 'No' 'No' 'No']


In [ ]:
x_train.shape

In [ ]:
x_train.columns

In [113]:
x_test.shape

(119606, 55)

In [114]:
x_train.columns

Index(['obj_to_date__date_diff', 'get_test__Sub_product_isNan',
       'get_test__Sub_issue_isNan',
       'get_test__Consumer_complaint_narrative_isNan',
       'get_test__Company_public_response_isNan', 'get_test__Tags_isNan',
       'get_test__Consumer_consent_provided_isNan',
       'get_test1__Issue_Loan_modification_collection_foreclosure',
       'get_test1__Issue_Incorrect_information_on_credit_report',
       'get_test1__Issue_Loan_servicing__payments__escrow_account',
       'get_test1__Issue_Cont'd_attempts_collect_debt_not_owed',
       'get_test1__Issue_Account_opening__closing__or_management',
       'get_test1__Issue_Disclosure_verification_of_debt',
       'get_test1__Issue_Communication_tactics',
       'get_test1__Issue_Deposits_and_withdrawals',
       'get_test1__Issue_Application__originator__mortgage_broker',
       'get_test1__Issue_Billing_disputes', 'get_test2__State_CA',
       'get_test2__State_FL', 'get_test2__State_TX', 'get_test2__State_NY',
       'get_te

In [112]:
test_columns= x_test.columns
train_columns = x_train.columns

for i in range(55):
        if test_columns[i]!=train_columns[i]:
            print("column : " + str(i))
            print(test_columns[i])
            
            print(train_columns[i])
            
     
        
            


In [ ]:
cd_train.head(10)


In [ ]:
test=k.axes[0][0:10]

In [ ]:
print(test)

In [ ]:
test1=k1.axes[0][0:15]
print(test1)
